<div id="header">
<h1 class="title"> Prêmio Dados Abertos para o Desenvolvimento - BNDES</h1>
<h2 class="author">Time 15</h2>
<h3 class="date">Agosto de 2021</h3>
</div>
<p><span style="font-variant: small-caps;">Time 15</span><br />
<span style="font-variant: small-caps;">E se o BNDES fosse um país?</span><br />
</p>
<hr />
<p><br />
<span> <strong></strong></span><br />
</p>
<hr />
<p><br />
</p>
<p><em>Autores:</em><br />
Caue Caviglioni<br />
Marcello Campanelli<br />
Pedro Xavier<br />
Rafael Ribeiro<br />
Ronald Albert<br />
</p>

<head>
  <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
  <meta http-equiv="Content-Style-Type" content="text/css" />
  <meta name="generator" content="pandoc" />
  <meta name="author" content="Time 15" />
  <title> Prêmio Dados Abertos para o Desenvolvimento - BNDES</title>
  <style type="text/css">code{white-space: pre;}</style>
</head>

In [ ]:
import pandas as pd
import seaborn as sns
import datetime
import matplotlib.pyplot as plt
import numpy as np
import zipfile
import plotly.graph_objects as go
import datetime
from sklearn.cluster import KMeans

<p><br />
</p>
<h1 id="introdução"> Introdução </h1>
<p>Com o avanço das tecnologias e a inserção do computador tanto no ambiente corporativo quanto no da população em geral, surgiu a possibilidade de se trabalhar com dados de maneira muito mais rápida e eficiente. Catalogar, processar e visualizar os dados deixaram de ser processos manuais e passaram a ser auxiliados por essas máquinas que revolucionaram o mundo.</p>
<p>Atualmente, o cada vez maior número de <em>data lakes</em> torna evidente a tendência de empresas e órgãos públicos a tornarem seus dados acessíveis ao público geral. Dessa forma, observamos a crescente valorização dos dados abertos e de instituições mais transparentes.</p>
<p>Aderindo a essa tendência, o Banco Nacional de Desenvolvimento Econômico e Social (<em>BNDES</em>) lançou seu <a href="https://www.bndes.gov.br/wps/portal/site/home/acesso-a-informacao/acesso-a-informacao-dados-abertos/!ut/p/z1/tVRNd6IwFN37K7phiUkkKJ0dIgUVba1fyMYTMGrmKKEh1XZ-_QSGzrSOtWdOOyx48Li5792bPEBUu7qq1UCognqobi8RRCk5sA2RjKdkB0KwiJrL4Lrv-vgWBt49RNA2-zf2rNM37IEJ5iUAvnPZEETqc2vkOb6NUWBNfAeOnJ56aXnI6yIwAxGIklRmcgsWcbqi-ZKluWTyMSk70OCW76kGSULznOtEZ-maiz1JCD-b1FdkxXOdxFRInhfkWcJWYNE0rDUyE6SvEFrrODGwHtMW0hutBGLcpDElsBJzQW10WWuvdPGPmYWdl-WH1YoK_juABfv-8BDZyhueSvokQdgedtzxsvvWHL80ZyJImmdE0DRhRIMsZSpKdiC5BjNB9-zUlnlhzGup0Li7gbYfoCketGGjj08Bt13DVQAnMLHbR961UQEuuLFQbrZeuXk3VCXMURt6LkbYx2B-YPQIpmmxdTsw_sfN8j-qMPpshVP6WbsD7aA7DrwpNCy38cX0J903_y-9-Un63tvz8feshGfO8NcPuKrSEANnsFHNE7ktgSD8eJ1qf7Pj8a9fnJ3GhqUYBF1TQUX9Uaj0Vsos_6ZBDR6Px3rZeX3DD_VYqExWTBYXsug_Z5K-qJDvTuK5OlueK1NO6UG2n-4t41mP4ua9_6M91D0nto6T9b4Kc0taz8Zu8xOEgb8o/dz/d5/L2dBISEvZ0FBIS9nQSEh/"><em>Plano de Dados Abertos</em></a>, com o objetivo de promover ganhos à administração pública. Motivados por isso, trazemos neste documento análises contendo diversos dados abertos, não sendo esses restritos ao <em>BNDES</em>.</p>
<h2 id="subsec:proposta">Proposta de Análise</h2>
<p>Avaliar a distribuição dos desembolsos do <em>BNDES</em> e comparar com diversos países, fazendo uso do crescimento percentual do <em>PIB</em> como parâmetro de sucesso econômico. Ao longo deste documento, vamos criar métricas de comparação a fim de traçar análises entre o <em>BNDES</em> e esses países referência, levantando questionamentos e insights.</p>
<p>Os países serão escolhidos com base em sua relevância econômica, bem como pela similaridade entre o <em>BNDES</em> e os países em questão. Essa similaridade será tratada utilizando algoritmos de clusterização, já que seria ineficiente e raso analisar os países de forma apenas manual.</p>
<h2 id="base-desembolsos-mensais-do-bndes">Base Desembolsos Mensais do BNDES</h2>
<p>Com a proposta acima, a base mais adequada a ser utilizada foi a de Desembolsos Mensais, que detalha os investimentos mensais do Sistema <em>BNDES</em>, por setores e por data, tendo uma cobertura temporal de 1995 até os meses atuais.</p>
<p>Essa base foi especialmente boa, pois foi possível comparar os desembolsos do <em>BNDES</em> com os investimentos de outros países, isso permitiu que o <em>BNDES</em> fosse tratado analogamente como um país.</p>


- [Base BNDES (Desembolsos)](https://dadosabertos.bndes.gov.br/dataset/desembolsos-mensais/resource/aac5c7bd-5d18-41c3-9720-52d20470427e) (Carregada por meio da API do portal de dados abertos do BNDES)


In [ ]:
url_desembolsos = "https://dadosabertos.bndes.gov.br/dataset/c529f6a7-5ce4-4378-a585-92cbf53af710/resource/aac5c7bd-5d18-41c3-9720-52d20470427e/download/desembolsos-mensais.csv"

df_desembolsos = pd.read_csv(url_desembolsos, sep=';',decimal=",")

In [ ]:
df_desembolsos.head(15)

<p>Informações interativas e mais detalhadas sobre a base podem ser encontradas no <a href="https://www.bndes.gov.br/wps/portal/site/home/transparencia/estatisticas-desempenho/estatisticas-operacionais-sistema-bndes"><em>dashboards de desembolsos</em> </a> do <em>BNDES</em>.</p>

<h2 id="subsec:dre">Base DRE - Série Histórica</h2>

<p>A base de <code>DRE - Série Histórica</code> trata da demonstração de diversos resultados do <em>BNDES</em>, e servirá de apoio à já citada criação da métrica de comparação, presente na Seção proposta. Utilizaremos ela como um paralelo ao <em>PIB</em> dos países comparados, como um parâmetro de sucesso ou insucesso econômico da instituição.</p>

<H>Carregamento da base e leve tratamento dos dados:

In [ ]:
url_balanco = "https://dadosabertos.bndes.gov.br/dataset/8d01af82-301b-4481-a2fc-3daf3a7bdf03/resource/159c6303-973d-413a-8de7-5343c318d37e/download/indicadores-financeiros-balanco-patrimonial-serie-historica.csv"
df_balanco = pd.read_csv(url_balanco, sep=';',decimal=",")
df_balanco['data'] = pd.to_datetime(df_balanco['data'])
df_balanco['ano'] = df_balanco.data.dt.year
df_balanco_interesse = df_balanco[["patrimonio_liquido", 'ano']].groupby('ano').mean()
pct_change = df_balanco_interesse.pct_change()
pct_change = pct_change * 100

<p>A base tem 77 registros e 35 colunas. As colunas de interesse da base são as de <code>data</code> e <code>patrimonio_liquido</code>, como mostradas na seção. A coluna referente ao patrimônio é tratada em milhões de reais.</p>

<h2 id="base-wdi">Base WDI</h2>
<p>Outra base que será usada é a base <code>WDI</code>, fornecida pelo The World Bank, essa que contém indicadores de diversas áreas relacionadas a gestão pública para inúmeros países.</p>
<p>A <code>WDI</code> é um <em>dataset</em> extremamente robusto. Para fins dessa análise, usaremos somente as colunas referentes a porcentagem de investimento em determinadas áreas e o crescimento anual do PIB de países.<br />
Abaixo, uma imagem da base com as colunas de porcentagem de investimento anual em químicos, indústria e crescimento anual do PIB</p>

- [Base WDI (Países)](https://datatopics.worldbank.org/world-development-indicators/) (Carregada localmente)

In [ ]:
wdi_zip = zipfile.ZipFile('../data/WDI_csv.zip')
df_wdi = pd.read_csv(wdi_zip.open('WDIData.csv'))

In [ ]:
df_wdi.head(3)

<p>A base foi coletada do <a href="https://databank.worldbank.org/source/world-development-indicators"><em>website do The World Bank</em></a>.</p>

<h1 id="análise-exploratória-dos-dados">Análise exploratória dos dados</h1>

<h2 id="base-desembolsos-mensais-do-bndes-1">Base Desembolsos Mensais do BNDES</h2>
<p>A base em questão tem <span class="math inline">3244804</span> entradas e <span class="math inline">16</span> colunas a serem analisadas. A descrição detalhada de cada coluna dessa base, já apresentada por alto na Figura [fig:base_bndes], pode ser encontrada no <a href="https://dadosabertos.bndes.gov.br/dataset/c529f6a7-5ce4-4378-a585-92cbf53af710/resource/d4cc2eda-8743-46de-a813-f9f1da018c0f/download/dicionario-de-dados-desembolsos-mensais-do-sistema-bndes.pdf"><em>portal de dados abertos do BNDES</em></a>.</p>
<p>Para o fim de nossa análise, vamos focar em uma granularidade temporal anual e de setores de investimento com base na coluna <code>setor_bndes</code>. Essas escolhas foram feitas visando tornar todas as três bases compatíveis.</p>
<p>Abaixo vemos a quantidade de investimentos do BNDES por setor:</p>

In [ ]:
desembolsosBySetor = pd.DataFrame(df_desembolsos.groupby(by=["ano", "setor_bndes"]).sum().desembolsos_reais).unstack()
desembolsosBySetor.columns = desembolsosBySetor.columns.levels[1]

sns.lineplot(data=desembolsosBySetor)

<p>Podemos observar pela figura que houve um considerável crescimento do investimento do BNDES entre os anos de 2006 e 2017 nas áreas de Infraestrutura, Comércio e Serviços e Indústria, no entanto, tal comportamento não foi observado em na área de Agropecuária.</p>
<p>Para melhor observar essa relação de do investimento entre os setores, temos a figura abaixo, a qual nos mostra um gráfico de percentual investido em cada um dos setores anualmente.

In [ ]:
porcentoSetor = desembolsosBySetor.div(desembolsosBySetor.sum(axis=1), axis=0)
porcentoSetor = porcentoSetor.apply(lambda x: x*100)

sns.lineplot(data=porcentoSetor)

<p>A partir do gráfico de percentuais vemos que os investimentos do BNDES, tem uma histórica preferencia pelos setores de Indústria e Infraestrutura. É possível observar também que a Agropecuária vinha perdendo espaço para os outros dois setores entre 2006 e 2015, no entanto, recuperando espaço nos últimos anos, a partir de 2018.

<h2 id="base-wdi-1">Base WDI</h2>
<p>Essa base possui dados de de muitos países, com diversos indicadores e com uma cobertura temporal de 1960 até 2020, no momento. No total, possui <span class="math inline">383838</span> registros e <span class="math inline">66</span> colunas, que vamos tratar com agrupamentos e filtros para extrair novas bases a serem utilizadas para cruzar com dados da base <em>BNDES</em>. Mais detalhes sobre essa extensa base estão <a href="https://datatopics.worldbank.org/world-development-indicators/"><em>na página do The World Bank</em></a>.</p>
<h1 id="tratamento-dos-dados-e-criação-de-uma-base-agregada">Tratamento dos dados e criação de uma base agregada</h1>
<p>A fim de criarmos análises sólidas de maneira mais rápida e simples, optamos por criar uma base agregada com informações de interesse presentes nas três bases anteriormente citadas.</p>
<p>Vale mencionar que o alicerce desse cruzamento de informações entre as duas bases foi a relação encontrada no portal de dados abertos do <em>BNDES</em>, na base <code>Mapeamento de BNDES para CNAE</code>, bem como a comparação com o <a href="https://datatopics.worldbank.org/world-development-indicators/"><em>dicionário de indicadores da base WDI</em></a>.</p>

![fluxograma_mapeamento.png](..\img\fluxograma_mapeamento.png)

## Tratandos os dados dos desembolsos

In [ ]:
df_desembolsos['date'] = pd.to_datetime(df_desembolsos['ano'].astype(str) + '-' + df_desembolsos['mes'].astype(str) + '-' + "01")
df_desembolsos.loc[:,['regiao', 'uf', 'municipio']] = df_desembolsos.loc[:,['regiao', 'uf', 'municipio']].replace('\s*$', '',regex=True)
df_desembolsos.sort_values(by=['date'], inplace=True)
df_desembolsos.head()

## Tratandos os dados da base WDI

In [ ]:
wdi_t = df_wdi.T
idx = pd.IndexSlice

wdi_t.columns = pd.MultiIndex.from_product([wdi_t.loc['Country Name'].unique(), wdi_t.loc['Indicator Name'].unique()])
wdi_t.drop(labels=['Country Name','Indicator Name', 'Country Code'], axis=0, inplace=True)
wdi_t.drop(index=['Indicator Code','Unnamed: 65'], inplace=True)
wdi_t.index = list(wdi_t.index.str.extract(r'([0-9]{4})').values.astype(int).reshape((1,-1)))
wdi_t.index = [i[0] for i in wdi_t.index.to_flat_index()]
wdi_t.index = wdi_t.index.astype(int)
wdi_t.replace(regex=r'^.*$', value=np.nan, inplace=True)
wdi_t = wdi_t.apply(pd.to_numeric)
wdi_t.head()

Vamos checar agora quais anos sao cobertos pelos dados do BNDES para filtrarmos a base WDI com dados de nosso interesse.

In [ ]:
lista_anos_interesse = df_desembolsos['ano'].sort_values().unique().astype(str)

print(lista_anos_interesse)

Vemos que sao dados a partir de 1995 com termino em 2020. Com essas informacoes, vamos filtrar as colunas da base wdi. Vamos manter as colunas contendo o nome do pais, codigo do pais e nome do indicador (Country Name, Country Code, Indicator Name)

In [ ]:
df_wdi[lista_anos_interesse]

Usamos o seguinte dicionário, para estabelecer a relação entre a base WDI e a base dos desembolsos do BNDES

In [ ]:
relacao_01 = {"quimica" : ["QUÍMICA E PETROQUÍMICA", "Chemicals (% of value added in manufacturing)"], 
              "agropecuaria": ["AGROPECUÁRIA", "Agriculture, forestry, and fishing, value added (% of GDP)"], 
              "industria": ["INDUSTRIA", "Industry (including construction), value added (% of GDP)"],
              "servicos":["COMÉRCIO E SERVIÇOS", "Services, value added (% of GDP)"] }


De posse do mapeamento entre os indicadores do *BNDES* e do WDI, chegamos à base agregada. Além disso, é importante ressaltar que, para que as análises fossem feitas em mesma escala, utilizamos o crescimento percentual do ano com seu antecessor como valor padrão para os indicadores tanto do *BNDES* quanto para o WDI.  

![corr.svg](..\img\corr.svg)

Acima, podemos ver a matriz de correlação para o investimento em algumas áreas do BNDES e o crescimento do PIB Brasileiro.<br>
Vemos que o único setor de investimento que possui uma correlação positiva com o PIB Brasileiro é o investimento em Indústria.

# Investimentos por setores de países e BNDES

Com dados sobre gastos em determinados setores presentes na base agregada, tanto do <i>BNDES</i> quanto o de outros países desenvolvidos, selecionamos três setores para comparação entre o <i>BNDES</i> e esses outros países: Indústria, agropecuária e serviços. <br><br>
    Esses setores e o de infraestrutura eram os mais relevantes, por isso escolhemos esses para análise. É importante mencionar que o setor de infraestrutura não foi utilizado, pois o mapeamento entre os subsetores de Infraestrutura da base do <i>BNDES</i> e da base <i>WDI</i> se tornou inviável, já que a relação entre a base do <i>BNDES</i> e do WDI não tinham uma correspondência clara.

Escolhemos alguns dos países mais desenvolvidos tecnologicamente para checar como o *BNDES* se comportou em relação a seus investimentos, se manteve uma postura parecida ou destoou no que os países mais desenvolvidos focaram.
<br><br> Os países escolhidos para efeitos de comparação do *BNDES*, foram o Brasil, China, Estados Unidos e Alemanha.  


In [ ]:
paises_interesse_p = ['Brazil', 'China', 'United States', 'Germany', 'BNDES']

Também escolhemos países relevantes mais relevantes no cenário internacional para checar similaridades com o *BNDES* usando Clusterização, para isso, foram selecionados 12 países: Brasil, Estados Unidos, China, Alemanha, Reino Unido, Japão, Itália, França, Canada, Australia, Argentina e Uruguai.

In [ ]:
paises_interesse = ['Brazil', 'United States', 'China', 'Germany', 'United Kingdom', 'Japan', 'Italy', 'France', 'Canada', 'Australia', 'BNDES', 'Argentina', 'Uruguay']

Abaixo podemos ver uma matriz de correlação das % de investimentos em alguns setores econômicos, com o crescimento do PIB brasileiro

In [ ]:
countries = ['Brazil']

sns.heatmap(
    wdi_t.loc[lista_anos_interesse.astype(int),idx[countries,[relacao_01['servicos'][1]] + [relacao_01['industria'][1]] + [relacao_01['agropecuaria'][1]] + ['GDP growth (annual %)']]].corr(),
    annot=True,
    linewidths=.5,
    xticklabels=['% de investimento em Serviços', '% de investimento em Indústria', '% de investimento em Agropecuária', 'PIB'], 
    yticklabels=['% de investimento em Serviços', '% de investimento em Indústria', '% de investimento em Agropecuária', 'PIB']
)

## Setor - Indústria

In [ ]:
setor = "industria"

aux_wdi = wdi_t[4:].loc[:,pd.IndexSlice[:, relacao_01[setor][1]]]
aux_wdi = aux_wdi.droplevel(1, axis=1)
aux_wdi = aux_wdi.fillna(0)
aux_wdi = aux_wdi.apply(lambda x: x.replace(0, x.iloc[np.array(x).nonzero()].mean()) if len(np.array(x).nonzero()[0]) > 0 else x
                        , axis=0)
aux_wdi = aux_wdi.loc[lista_anos_interesse.astype(int)]

concat = pd.concat([aux_wdi, pd.DataFrame({'BNDES': porcentoSetor[relacao_01[setor][0]]})], axis=1)
concat.head()

In [ ]:
sns.lineplot(data=concat.loc[lista_anos_interesse.astype(int), paises_interesse_p])

Analisando o gráfico acima, vemos que os países mais desenvolvidos investem boa parte no setor da indústria, enquanto o *BNDES* se mantém muito abaixo de todos em relação aos mesmos desembolsos.

<br> Com a clusterização, vamos comparar o *BNDES* com países relevantes no contexto internacional para checar com quais o *BNDES* vai se assimilar no setor de indústria.

In [ ]:
concat = concat[paises_interesse]

In [ ]:
#ANALISE DO NUMERO DE CLUSTER
def cidade_grupo(nclusters, pais='BNDES'):
    pais_index = list(concat.columns).index(pais) 
    lista_paises = list()
    
    for _ in range(20):
        clustering = KMeans(n_clusters=nclusters,algorithm='full').fit(concat.values.T)
        lista_paises += (list(concat.columns[np.where(clustering.labels_ == clustering.labels_[pais_index])[0]]))
        
    return set(lista_paises)

paises_similares_dict = {} 
ncls = []
n_clusters = np.array(range(2,6))
for k in n_clusters:
    listpaises = cidade_grupo(k)
    paises_similares_dict[str(k)] = list(listpaises)
    ncls.append(len(listpaises))
    
plt.bar(range(len(ncls)),ncls)
plt.xticks(range(len(n_clusters)), range(n_clusters.min(),n_clusters.max() + 1))
plt.title('Tot. de países no cluster do BNDES x num. de clusters')
plt.xlabel('Numero de clusters')
plt.ylabel('Total de paises no cluster do BNDES')
plt.savefig('../img/n_clusters_'+setor+'.svg')
plt.show()

Vemos que o *BNDES* se agrupa no máximo com 1 país, ao ser clusterizado com 2 clusters. 

<br> Dentre os 12, os países que se agruparam com o *BNDES* nesse setor foram , segundo o gráfico:

In [ ]:
countries = np.array(paises_similares_dict['2'])

sns.lineplot(data = concat[countries])

Nessa clusterização, o *BNDES* se agrupou com a China.

## Setor - Agropecuária

In [ ]:
setor = "agropecuaria"

aux_wdi = wdi_t[4:].loc[:,pd.IndexSlice[:, relacao_01[setor][1]]]
aux_wdi = aux_wdi.droplevel(1, axis=1)
aux_wdi = aux_wdi.fillna(0)
aux_wdi = aux_wdi.apply(lambda x: x.replace(0, x.iloc[np.array(x).nonzero()].mean()) if len(np.array(x).nonzero()[0]) > 0 else x
                        , axis=0)
aux_wdi = aux_wdi.loc[lista_anos_interesse.astype(int)]

concat = pd.concat([aux_wdi, pd.DataFrame({'BNDES': porcentoSetor[relacao_01[setor][0]]})], axis=1)
concat.head()

In [ ]:
sns.lineplot(data=concat.loc[lista_anos_interesse.astype(int), paises_interesse_p])

<p>Com esse gráfico relativo ao investimento percentual no setor de agropecuária, vemos que a tendência dos países mais desenvolvidos se manteve relativamente estável, ou desceu ao longo dos anos. O <em>BNDES</em> não acompanhou tão bem os países mais desenvolvidos, investindo muito em agropecuária. Porém, no gráfico é possível perceber a relevância do <em>BNDES</em> para o Brasil, já que quando há uma subida súbita no gráfico do <em>BNDES</em>, o percentual investido do Brasil também sobe.</p>

<p>Na Figura abaixo, vemos que a correlação dos desembolsos e a agropecuária é negativo <span class="math inline">( − 0.37)</span>, o que é considerado uma correlação fraca. Isso significa que, para o período analisado, o aumento nos investimentos por parte do <em>BNDES</em> no setor de agropecuária esteve relacionado a um decrescimento do PIB.</p>
<p>Vale notar que, pelo PIB ser composto por muitos outros fatores além dos resultados financeiros advindos da agropecuária, essa análise não pode ser generalizada. Em outras palavras, o aumento no investimento da agropecuária não necessariamente acarreta uma diminuição do PIB.</p>


In [ ]:
paises = ['Brazil', 'Patrimônio BNDES', 'PIB Brasileiro']

concat_gdp = pd.concat([concat, pct_change, wdi_t.loc[lista_anos_interesse.astype(int),idx[['Brazil'], ['GDP growth (annual %)']]]], axis=1)
col = list(concat_gdp.columns[:-1])
col.append('PIB Brasileiro')
concat_gdp.columns = col
concat_gdp.rename(columns = {'patrimonio_liquido': 'Patrimônio BNDES'}, inplace = True)

corr_df = concat_gdp.loc[lista_anos_interesse.astype(int), paises]
corr_df = pd.concat([corr_df, pd.DataFrame({'BNDES - Agropecuária': porcentoSetor['AGROPECUÁRIA']})], axis=1)
corr_df = corr_df[['BNDES - Agropecuária', 'Brazil','Patrimônio BNDES', 'PIB Brasileiro']]
sns.heatmap(corr_df.corr(),
    annot=True,
    linewidths=.5)

plt.savefig('../img/corr.svg')

<p>Selecionamos alguns países relevantes no contexto internacional para clusterizar junto com o <em>BNDES</em>.<br> O resultado está na Figura :</p>

In [ ]:
concat = concat[paises_interesse]

In [ ]:
#ANALISE DO NUMERO DE CLUSTER
def cidade_grupo(nclusters, pais='BNDES'):
    pais_index = list(concat.columns).index(pais) 
    lista_paises = list()
    
    for _ in range(20):
        clustering = KMeans(n_clusters=nclusters,algorithm='full').fit(concat.values.T)
        lista_paises += (list(concat.columns[np.where(clustering.labels_ == clustering.labels_[pais_index])[0]]))
        
    return set(lista_paises)

paises_similares_dict = {} 
ncls = []
n_clusters = np.array(range(2,6))
for k in n_clusters:
    listpaises = cidade_grupo(k)
    paises_similares_dict[str(k)] = list(listpaises)
    ncls.append(len(listpaises))
    
plt.bar(range(len(ncls)),ncls)
plt.xticks(range(len(n_clusters)), range(n_clusters.min(),n_clusters.max() + 1))
plt.title('Tot. de países no cluster do BNDES x num. de clusters')
plt.xlabel('Numero de clusters')
plt.ylabel('Total de paises no cluster do BNDES')
plt.savefig('../img/n_clusters_'+setor+'.svg')
plt.show()

<p>Com 2 Clusters, o <em>BNDES</em> se agrupou com 3 outros países, com mais que 2, o <em>BNDES</em> não conseguiu se assimilar a nenhum país.</p>
<p>Os países que se agruparam com o <em>BNDES</em> nesse setor foram:</p>
<p>O <em>BNDES</em> se assimila à China, Argentina e ao Uruguai.</p>

In [ ]:
countries = np.array(paises_similares_dict['2'])

sns.lineplot(data = concat[countries])

## Setor - Serviços

In [ ]:
setor = "servicos"

aux_wdi = wdi_t[4:].loc[:,pd.IndexSlice[:, relacao_01[setor][1]]]
aux_wdi = aux_wdi.droplevel(1, axis=1)
aux_wdi = aux_wdi.fillna(0)
aux_wdi = aux_wdi.apply(lambda x: x.replace(0, x.iloc[np.array(x).nonzero()].mean()) if len(np.array(x).nonzero()[0]) > 0 else x
                        , axis=0)
aux_wdi = aux_wdi.loc[lista_anos_interesse.astype(int)]

concat = pd.concat([aux_wdi, pd.DataFrame({'BNDES': porcentoSetor[relacao_01[setor][0]]})], axis=1)
concat.head()

In [ ]:
sns.lineplot(data=concat.loc[lista_anos_interesse.astype(int), paises_interesse_p])

No setor de serviços, os países mais desenvolvidos se mantiveram estáveis, com exceção da China, que veio se desenvolvendo e incrementando os investimentos em serviços. O *BNDES* teve um comportamento interessante, e de certa forma acompanhou o crescimento dos investimentos da China no determinado setor, o que é otimista, pois a China é o país que mais cresce tecnologicamente e economicamente nos tempos atuais.
 

Nas células abaixo, vamos clusterizar com os países de interesse para ver com quais o \textit{BNDES} se agrupa.

In [ ]:
concat = concat[paises_interesse]

In [ ]:
#ANALISE DO NUMERO DE CLUSTER
def cidade_grupo(nclusters, pais='BNDES'):
    pais_index = list(concat.columns).index(pais) 
    lista_paises = list()
    
    for _ in range(20):
        clustering = KMeans(n_clusters=nclusters,algorithm='full').fit(concat.values.T)
        lista_paises += (list(concat.columns[np.where(clustering.labels_ == clustering.labels_[pais_index])[0]]))
        
    return set(lista_paises)

paises_similares_dict = {} 
ncls = []
n_clusters = np.array(range(2,6))
for k in n_clusters:
    listpaises = cidade_grupo(k)
    paises_similares_dict[str(k)] = list(listpaises)
    ncls.append(len(listpaises))
    
plt.bar(range(len(ncls)),ncls)
plt.xticks(range(len(n_clusters)), range(n_clusters.min(),n_clusters.max() + 1))
plt.title('Tot. de países no cluster do BNDES x num. de clusters')
plt.xlabel('Numero de clusters')
plt.ylabel('Total de paises no cluster do BNDES')
plt.savefig('../img/n_clusters_'+setor+'.svg')
plt.show()

Nessa clusterização sobre o setor de serviços ele ficou sozinho no agrupamento. <br>
Como vemos na imagem abaixo:

In [ ]:
countries = np.array(paises_similares_dict['2'])

sns.lineplot(data = concat[countries])